In [ ]:
import s3fs, pyarrow.parquet as pq, pandas as pd


In [ ]:

fs = s3fs.S3FileSystem(
    key="minioadmin", secret="minioadmin",
    client_kwargs={"endpoint_url": "http://localhost:9000"}
)

prefix = "deltabucket/silver/wholeCorp_delta"
files = fs.glob(f"{prefix}/*.parquet")

dfs = []
for f in files:
    table = pq.ParquetDataset(f"s3://{f}", filesystem=fs).read()
    df = table.to_pandas()
    dfs.append(df)

df_whole_silver = pd.concat(dfs, ignore_index=True)

In [79]:
prefix = "deltabucket/gold/wholeCorp_delta"
files = fs.glob(f"{prefix}/*.parquet")

dfs = []
for f in files:
    table = pq.ParquetDataset(f"s3://{f}", filesystem=fs).read()
    df = table.to_pandas()
    dfs.append(df)

df_whole_gold = pd.concat(dfs)

df_merged = df_whole_silver.merge(df_whole_gold[['統一編號','features']], on= "統一編號", how='left')
df_merged = df_merged[['統一編號', '公司名稱', '負責人', '登記地址', '資本額', '營業項目及代碼表', '縣市名稱', '區域名稱',
        '類別_全', '官網', '電話', 'features']]
df_merged.rename(columns={'features':'features_vector'}, inplace=True)
df_merged

,統一編號,公司名稱,負責人,登記地址,資本額,營業項目及代碼表,縣市名稱,區域名稱,類別_全,官網,電話,features_vector
0,37838867,勁霸平價熱炒,廖曉雯,宜蘭縣羅東鎮仁愛里仁愛路三段66號,1000000.0,餐館業,宜蘭縣,羅東鎮,餐館業,\N,\N\r,"{'type': 0, 'size': 262147, 'indices': [178355..."
1,37838873,北海讚日式涮涮鍋,吳榮華,宜蘭縣羅東鎮漢民里純精路二段126號一樓,2000000.0,"餐館業,服飾品零售業",宜蘭縣,羅東鎮,"餐館業,服飾品零售業",\N,\N\r,"{'type': 0, 'size': 262147, 'indices': [34950,..."
2,37839079,弘宸精品火鍋城,李明宸,宜蘭縣羅東鎮仁愛里復興路三段542號,800000.0,餐館業,宜蘭縣,羅東鎮,餐館業,\N,\N\r,"{'type': 0, 'size': 262147, 'indices': [178355..."
3,37839768,茉尼好食光早午餐,陳名儀,宜蘭縣五結鄉五結村中興路一段37號一樓,2000000.0,餐館業,宜蘭縣,五結鄉,餐館業,\N,\N\r,"{'type': 0, 'size': 262147, 'indices': [178355..."
4,37840792,溫泉快炒小站,陳玉芬,宜蘭縣礁溪鄉玉田村玉龍路一段57號一樓,500000.0,餐館業,宜蘭縣,礁溪鄉,餐館業,\N,\N\r,"{'type': 0, 'size': 262147, 'indices': [178355..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1379781,20060883,上豪汽車電機行,\N,花蓮縣吉安鄉慶豐村慶豐10街126巷13號,NaN,\N,花蓮縣,吉安鄉,工商專業服務/金融工商/車輛輪胎保修,\N,\N\r,"{'type': 0, 'size': 262147, 'indices': [195778..."
1379782,80974285,汽車電持，機車電池，相機電池，不斷電系統,\N,\N,NaN,\N,\N,\N,工商專業服務/金融工商/車輛輪胎保修,\N,04-26521546\r,"{'type': 0, 'size': 262147, 'indices': [195778..."
1379783,28605402,汽車維修與報廢汽機車回收,\N,\N,NaN,\N,\N,\N,工商專業服務/金融工商/車輛輪胎保修,\N,08-8682859\r,"{'type': 0, 'size': 262147, 'indices': [195778..."
1379784,13513101,和撒那汽車商行,\N,新北市新店區安和路3段142號(1樓),NaN,\N,新北市,新店區,工商專業服務/金融工商/車輛輪胎保修,\N,02-29420014\r,"{'type': 0, 'size': 262147, 'indices': [195778..."


In [ ]:
df_merged.features_vector[0]

{'type': 0,
 'size': 262147,
 'indices': array([178355, 262144, 262145, 262146], dtype=int32),
 'values': array([5.21230542, 2.32432391, 0.        , 0.        ])}

: 

In [ ]:
# convert "features" struct into dense numpy vector
def to_dense(row):
    if pd.isna(row):
        return None
    indices = row["indices"]
    values = row["values"]
    size = row["size"]
    dense = np.zeros(size, dtype=float)
    dense[indices] = values
    return dense.tolist()dd


df_merged["features_vector"] = df_merged["features_vector"].apply(to_dense)
print(df_merged[["公司名稱","features_vector"]].head())


MemoryError: 

In [ ]:
# --- Read Delta from MinIO ---
storage_options = {
    "AWS_ACCESS_KEY_ID": "minioadmin",
    "AWS_SECRET_ACCESS_KEY": "minioadmin",
    "AWS_ENDPOINT_URL": MINIO_ENDPOINT,
}

dt = DeltaTable(DELTA_PATH, storage_options=storage_options)

# Convert to Pandas (you can chunk if too big)
df = dt.to_pandas()

# --- Send to Elasticsearch ---
es = Elasticsearch(ES_URL)

def docs():
    for rec in df.to_dict(orient="records"):
        yield {"_index": ES_INDEX, "_source": rec}

helpers.bulk(es, docs())
print(f"Inserted {len(df)} records into {ES_INDEX}")


ModuleNotFoundError: No module named 'elasticsearch'

# Check log

In [ ]:
import s3fs, json

fs = s3fs.S3FileSystem(
    key="minioadmin",
    secret="minioadmin",
    client_kwargs={"endpoint_url": "http://localhost:9000"},
)

log_path = "deltabucket/gold/wholeCorp_clusters/_delta_log/00000000000000000000.json"

In [ ]:
with fs.open(log_path) as f:
    for line in f:
        print(line)
        obj = json.loads(line)

b'{"commitInfo":{"timestamp":1756348865596,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"3","numOutputRows":"1379786","numOutputBytes":"50111755"},"engineInfo":"Apache-Spark/3.5.0 Delta-Lake/3.1.0","txnId":"c5fc9746-d3b9-4f5e-bf3a-b0c9fd0da940"}}\n'
b'{"metaData":{"id":"39abccd6-8638-43ab-a593-82213368be39","format":{"provider":"parquet","options":{}},"schemaString":"{\\"type\\":\\"struct\\",\\"fields\\":[{\\"name\\":\\"\xe7\xb5\xb1\xe4\xb8\x80\xe7\xb7\xa8\xe8\x99\x9f\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"\xe5\x85\xac\xe5\x8f\xb8\xe5\x90\x8d\xe7\xa8\xb1\\",\\"type\\":\\"string\\",\\"nullable\\":true,\\"metadata\\":{}},{\\"name\\":\\"features\\",\\"type\\":{\\"type\\":\\"udt\\",\\"class\\":\\"org.apache.spark.ml.linalg.VectorUDT\\",\\"pyClass\\":\\"pyspark.ml.linalg.VectorUDT\\",\\"sqlType\\":{\\"type\\":\\"struct\\",\\"fiel

In [ ]:
with fs.open(log_path) as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)
        if "metaData" in obj:
            print("Schema string:")
            print(obj["metaData"]["schemaString"])

Schema string:
{"type":"struct","fields":[{"name":"統一編號","type":"string","nullable":true,"metadata":{}},{"name":"公司名稱","type":"string","nullable":true,"metadata":{}},{"name":"features","type":{"type":"udt","class":"org.apache.spark.ml.linalg.VectorUDT","pyClass":"pyspark.ml.linalg.VectorUDT","sqlType":{"type":"struct","fields":[{"name":"type","type":"byte","nullable":false,"metadata":{}},{"name":"size","type":"integer","nullable":true,"metadata":{}},{"name":"indices","type":{"type":"array","elementType":"integer","containsNull":false},"nullable":true,"metadata":{}},{"name":"values","type":{"type":"array","elementType":"double","containsNull":false},"nullable":true,"metadata":{}}]}},"nullable":true,"metadata":{"ml_attr":{"num_attrs":262147}}}]}


# Read parquet cluster gold

In [77]:
prefix = "deltabucket/gold/wholeCorp_clusters"
files = fs.glob(f"{prefix}/*.parquet")

dfs = []
for f in files:
    table = pq.ParquetDataset(f"s3://{f}", filesystem=fs).read()
    df = table.to_pandas()
    dfs.append(df)

df_whole_gold_cluster = pd.concat(dfs)

# df_merged = df_whole_silver.merge(df_whole_gold[['統一編號','features']], on= "統一編號", how='left')
# df_merged = df_merged[['統一編號', '公司名稱', '負責人', '登記地址', '資本額', '營業項目及代碼表', '縣市名稱', '區域名稱',
#         '類別_全', '官網', '電話', 'features']]
# df_merged.rename(columns={'features':'features_vector'}, inplace=True)

In [78]:
df_whole_gold_cluster

,統一編號,cluster
0,72517384,5
1,72528994,2
2,72529494,13
3,72534187,13
4,72538389,4
...,...,...
344517,72506083,12
344518,72506751,12
344519,72507251,12
344520,72509758,0
